# KD值作為買進賣出的依據

從資料庫中取得資料，並根據KD黃金交叉來判斷何時買進、賣出，\
當黃金交叉時，依照當天收盤價做買進，\
死亡交叉時，依照當天收盤價做賣出，\
其中並無計算交易摩擦之成本以及手續費


In [1]:
%matplotlib inline 

import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import talib

In [38]:
# 從資料庫中取得資料

con = sqlite3.connect('mydata.db')

#num為取幾筆
def get(stock_id, num):
    df = pd.read_sql('SELECT * from price WHERE stock_id="0050" ',con)
    #df = df.drop(columns=['stock_id','stock_name'])
    df.date = pd.to_datetime(df.date)
    df = df.iloc[(-1)*num:]
    df.set_index(df.date,inplace=True, drop=True) 
    return df

In [77]:
df = get('0050',800)

In [78]:
df

,stock_id,stock_name,date,volume,open,high,low,close,漲跌,成交筆數,成交金額,最後買價,最後賣價
date,,,,,,,,,,,,,
2018-06-01,0050,元大台灣50,2018-06-01,1848694.0,80.80,81.45,80.80,81.35,+0.6,818.0,1.501415e+08,81.30,81.35
2018-06-04,0050,元大台灣50,2018-06-04,7081064.0,81.80,82.60,81.80,82.60,+1.25,2675.0,5.831084e+08,82.60,82.65
2018-06-05,0050,元大台灣50,2018-06-05,2695921.0,82.70,82.70,82.25,82.50,-0.1,1361.0,2.223384e+08,82.50,82.55
2018-06-06,0050,元大台灣50,2018-06-06,6398322.0,82.65,83.35,82.65,83.30,+0.8,2988.0,5.314562e+08,83.25,83.30
2018-06-07,0050,元大台灣50,2018-06-07,6704318.0,83.50,83.60,83.00,83.45,+0.15,1499.0,5.578493e+08,83.45,83.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-02,0050,元大台灣50,2021-09-02,7218178.0,140.90,141.40,139.90,139.90,-1.1,6517.0,1.015443e+09,139.9,139.95
2021-09-03,0050,元大台灣50,2021-09-03,11543959.0,140.65,142.10,140.45,141.90,+2.0,10027.0,1.634418e+09,141.90,141.95
2021-09-06,0050,元大台灣50,2021-09-06,14283400.0,141.90,143.50,141.60,142.55,+0.65,14230.0,2.037715e+09,142.50,142.60


### 買進訊號(ＫＤ黃金交叉）、賣出訊號(KD死亡交叉)

In [79]:
kd = talib.STOCH(df.high,df.low,df.close,fastk_period=9, slowk_period=3)

k = kd[0]
d = kd[1]

In [80]:
buy = (k.shift(1)<d.shift(1)) & (k>d) 
sell = (d.shift(1)<k.shift(1)) & (k<d) 
buy = pd.Series(buy, name='buy')
sell = pd.Series(sell, name='sell')

In [81]:
newdf = pd.concat([df,buy,sell],axis=1)

In [82]:
print(f"總共買進{len(newdf[newdf['buy']==True])}次")
print(f"總共賣出{len(newdf[newdf['sell']==True])}次")


總共買進86次
總共賣出86次


In [88]:
buydf = newdf[newdf.buy==True]
selldf = newdf[newdf.sell==True]

In [116]:
rr=[]
for i in range(len(buydf)):
    r = (float(selldf.close[i]) - float(buydf.close[i]))/float(buydf.close[i])*100
    rr.append(round(r,2))
    

win, lose =0,0
for i in rr:
    if i >0:
        win+=1
    else:
        lose+=1

In [136]:
print(f"最大虧損{np.min(rr)}％")
print(f"最大報酬率{np.max(rr)}％")
print(f"獲利次數{win}次")
print(f"虧損次數{lose}次")
print(f"平均報酬率{round(np.average(rr),2)}%")

最大虧損-7.59％
最大報酬率6.71％
獲利次數44次
虧損次數42次
平均報酬率0.59%
